In [1]:
import os
import re
import pandas as pd
import numpy as np
from natsort import natsorted, index_natsorted, order_by_index
from tools import * # функции для обработки
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)

### Обработка файлов формы № 10.3

In [2]:
year = 2019

Первый лист .xls-файла мы пропускаем, так как во всех отчетных формах там содержится общая информация, а не данные.

In [3]:
mailFile = 'ExcelFiles/Main/' + str(year) + 'main.xls'
mainDF = pd.read_excel(mailFile, 1, header=None)

После этого удаляем "шапку" таблицы со служебной информацией, колонки, не содержащие номера статьи или значения показателя (пустые и колонки "№ п/п" и "№ стр.")

In [4]:
mainDF = dropFirstRows(mainDF, 'Статьи УК РФ')

/home/kate/dostoevsky-data/tools.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, val in df.iloc[:,0].str.contains(value2search).iteritems():


In [5]:
mainDF = deleteUnusedCols(mainDF, year)

/home/kate/dostoevsky-data/tools.py:67: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, col in df.iloc[0:header_rows].iteritems():


Удаляем строку "Всего по составам УК РФ" и строку с номерами колонок. Значения строки "Всего по составам УК РФ" сохраняем в отдельную переменную - они будут нужны позже, для проверки корректности обработки данных.

In [6]:
total_values = list(mainDF[mainDF.iloc[:,0].astype('str').str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][2:])
mainDF = dropFirstRows(mainDF, '105 ч. 1|105 ч.1|105ч. 1', 2)

/home/kate/dostoevsky-data/tools.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, val in df.iloc[:,0].str.contains(value2search).iteritems():


Теперь разбиваем объединенные ячейки (если они есть), не теряя их содержимого: по каждому столбцу собираем текст, который распределен по объединенным ячейкам, в заголовок таблицы

In [7]:
mainDF.loc[1, [15, 16, 17, 18, 19, 20]] = mainDF.loc[1, [15, 16, 17, 18, 19, 20]].fillna(method='ffill')
mainDF = tackleMergedCells(mainDF)

/home/kate/dostoevsky-data/tools.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[0:1].fillna(method='ffill', axis='columns', inplace=True)
/home/kate/dostoevsky-data/tools.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[0:2].replace(to_replace=['\n', '-', '\*', '\s{3}', '\s{2}'],value=[' ', '', '', ' ', ' '], regex=True, inplace=True)


Чистим заголовок таблицы от лишних пробелов, переносов строки и прочего, переводим в нижний регистр

In [8]:
mainDF = cleanСolsNames(mainDF)

Переводим заголовок на английский. Английские названия для колонок лежат в отдельных файлах:    

- colNames2engNames.csv
- colNames2engNamesAdd.csv
- colNames2engNamesParameters.csv

Если колонка не была переименована, ее название печатается. Колонки с русскоязычными названиями далее будут удалены. В некоторых случаях так и должно быть: если колонка дублируется в исходных таблицах Судебного департамента, в своих данных мы используем ее только один раз.

In [9]:
columns = mainDF.columns # эта строка тоже будет нужна позже для проверки

columns2eng = pd.read_csv('colNames2engNames.csv')
columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
mainDF = renameColumns(mainDF, columns2eng)

Вырезаем первую строку и строки, в которых остались ячейки с пропущенными значениями (NA). Это либо отработанные строки заголовка, либо "подвал" внизу таблицы

In [10]:
mainDF = dropNARows(mainDF)

In [11]:
mainDF['clause'] = mainDF['clause'].astype('str')
mainDF['clause'] = mainDF['clause'].str.replace("Составы преступлений, \nвведенные в УК РФ в 2019 году:\n124.1 ч. 1", "124.1 ч. 1")

mainDF = deleteUselessRows(mainDF, year)

/tmp/ipykernel_296840/38749100.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  mainDF['clause'] = mainDF['clause'].str.replace("Составы преступлений, \nвведенные в УК РФ в 2019 году:\n124.1 ч. 1", "124.1 ч. 1")


In [12]:
mainDF = cleanClauseCol(mainDF) # чистим колонку с номерами статей от лишнего текста
mainDF = clauses2column(mainDF) # используя регулярное выражение, создаем колонки со номерами статей и частей 
mainDF = keepCombinedRows(mainDF, year) # переименовываем те номера статей, которые должны остаться объединенными, 
# т.к. они пострадали после предыдущей операции
firstCols = ['clause', 'part', 'totalConvicted']
mainDF = rearrangeCols(mainDF, firstCols) # меняем порядок столбцов для удобства

# поскольку названия частей вместе с примечаниями мы берем из формы 10-а, в тех случаях, где номера статей в формах 
# 10.3 и 10.3.1 и 10-а расходятся (в одной таблице - номер устаревшей статьи, а в друой - уже новой)
# - ориентируемся на 10-а

mainDF['part'] = mainDF['part'].replace('136ч.2', '136')
mainDF['part'] = mainDF['part'].replace('168$', '168ч.1', regex=True)

mainDF = sortTable(mainDF) # упорядочиваем строки

/home/kate/dostoevsky-data/tools.py:159: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в УК РФ после утверждения форм отчетности:\nКлевета \(введ. ФЗ от 28.07.2012 N 141-ФЗ\) ст. ", "")
/home/kate/dostoevsky-data/tools.py:160: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в раздел IX \"Преступления против общественной безопасности и общественного порядка\" УК РФ после утверждения форм отчетности приказом № 127\s+ст. ", "")
/home/kate/dostoevsky-data/tools.py:161: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Клевета в отношении судьи, присяжного заседателя, прокурора, следователя, лица, производящего дознание, судебного пристава \(введ. ФЗ от 28.07.2012 N

Выше мы сохранили значения строки "Всего по составам УК РФ", которая содержит сумммы по каждому столбцу. Теперь мы используем их для проверки корректности обработки данных, сравнив суммы по столбцам со значениями, указанными в строке "Всего по составам УК РФ".

К сожалению, как показала практика, числа эти не всегда совпадают, поэтому каждое несовпадение мы перепроверили вручную. Кроме того, еще в одном случае суммы действительно не совпадают из-за удаления в файлах 2011 год строки "Преступления, введенные в УК РФ после утверждения форм отчетности приказом № 115 от 10.06.2011" с одним осужденным к ограничению свободы (см. выше).

In [13]:
# выводит несоответствия сумм, если ничего не печатает - несоответствий нет
compareSums(mainDF, columns, total_values, 2)

### Обработка файлов формы № 10.3.1

In [14]:
parametersFile = 'ExcelFiles/MainParameters/' + str(year) + 'parameters.xls'
parametersDF = pd.read_excel(parametersFile, 1, header=None)

total_values = list(parametersDF[parametersDF.iloc[:,0].astype('str').str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][3:])

parametersDF = dropFirstRows(parametersDF, 'Статьи УК РФ')
parametersDF = deleteUnusedCols(parametersDF, year)
parametersDF = dropFirstRows(parametersDF, '105 ч. 1|105 ч.1|105ч. 1', 2)

parametersDF = tackleMergedCells(parametersDF)

parametersDF = cleanСolsNames(parametersDF)
columns2eng = pd.read_csv('colNames2engNamesParameters.csv')
columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
parametersDF = renameColumns(parametersDF, columns2eng)

columns = parametersDF.columns

parametersDF = parametersDF[parametersDF.columns[~parametersDF.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
parametersDF = dropNARows(parametersDF)

parametersDF['clause'] = parametersDF['clause'].astype('str')
parametersDF['clause'] = parametersDF['clause'].str.replace("Составы преступлений, \nвведенные в УК РФ в 2019 году:\n124.1 ч. 1", "124.1 ч.1")

parametersDF = deleteUselessRows(parametersDF, year)

parametersDF = cleanClauseCol(parametersDF)

parametersDF = solveProblem2012(parametersDF, year)

parametersDF = clauses2column(parametersDF)
parametersDF = solveProblem2013_2014(parametersDF, year)
parametersDF = keepCombinedRows(parametersDF, year)

parametersDF['part'] = parametersDF['part'].replace('136ч.2', '136')
parametersDF['part'] = parametersDF['part'].replace('168$', '168ч.1', regex=True)

firstCols = ['clause', 'part']
parametersDF = rearrangeCols(parametersDF, firstCols)
parametersDF = sortTable(parametersDF)

/home/kate/dostoevsky-data/tools.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, val in df.iloc[:,0].str.contains(value2search).iteritems():
/home/kate/dostoevsky-data/tools.py:67: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, col in df.iloc[0:header_rows].iteritems():
/home/kate/dostoevsky-data/tools.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, val in df.iloc[:,0].str.contains(value2search).iteritems():
/home/kate/dostoevsky-data/tools.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[0:1].fillna(method='ffill', axis='columns', inplace=True)
/home/kate/dostoevsky-data/tools.py:37:

лишение свободы всего (число лиц)
общая сумма штрафов (из гр. 13 "основное наказание" ф. 10.3) всего лиц
общая сумма штрафов (из гр. 28 "дополнительное наказание" ф. 10.3) всего лиц
общая сумма судебных штрафов, назначенных в соответствии со статьей 104.4 ук рф всего лиц


/tmp/ipykernel_296840/2574252099.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  parametersDF['clause'] = parametersDF['clause'].str.replace("Составы преступлений, \nвведенные в УК РФ в 2019 году:\n124.1 ч. 1", "124.1 ч.1")
/home/kate/dostoevsky-data/tools.py:159: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в УК РФ после утверждения форм отчетности:\nКлевета \(введ. ФЗ от 28.07.2012 N 141-ФЗ\) ст. ", "")
/home/kate/dostoevsky-data/tools.py:160: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в раздел IX \"Преступления против общественной безопасности и общественного порядка\" УК РФ после утверждения форм отчетности приказом № 127\s+ст. ", "")
/home/kate/dostoevsky-data/tools.py:161: FutureW

Снова проверяем суммы по столбцам.

In [15]:
compareSums(parametersDF, columns, total_values, 2)


Сумма значений в колонке primaryFineSum 3266481113 
 Значение в строке "Всего по составам УК РФ"  3266503113

Сумма значений в колонке addFineSum 3649376159 
 Значение в строке "Всего по составам УК РФ"  3649916159

Сумма значений в колонке dismissalCourtFine5_25 30311 
 Значение в строке "Всего по составам УК РФ"  30307

Сумма значений в колонке dismissalCourtFineSum 824971558 
 Значение в строке "Всего по составам УК РФ"  824979558


### Обработка файлов формы № 10-а 

In [16]:
names = {'263ч.1.2': 'Деяния, предусмотренные частями первой и первой.1 настоящей статьи, совершенные лицом, находящимся в состоянии опьянения, повлекшие по неосторожности причинение тяжкого вреда здоровью человека'
        }

addFile = 'ExcelFiles/Add/' + str(year) + 'add.xls'
if year >= 2017:
    df_list = pd.read_excel(addFile, sheet_name=None, header = None)
    dfs = []
    for key in list(df_list.keys())[2:-1]:
        df = df_list[key]
        df = dropFirstRows(df, 'Виды преступлений')
        df = dropFirstRows(df, '[а-яА-Я]{3,}', 1)
        df = deleteUnusedCols(df, year)
        df = tackleMergedCells(df)
        columns2eng = pd.read_csv('colNames2engNamesAdd.csv')
        columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
        df = renameColumns(df, columns2eng) # тут заголовок таблицы предварительно чистить не надо

        df = df[df.columns[~df.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
        df = dropNARows(df)
        dfs.append(df)
    addDF = pd.concat(dfs, ignore_index=True)
else:
    addDF = pd.read_excel(addFile, 1, header=None)
    addDF = dropFirstRows(addDF, 'Виды преступлений')
    addDF = dropFirstRows(addDF, 'ВСЕГО ПО РАЗДЕЛАМ', 1)
    addDF = deleteUnusedCols(addDF, year)
    addDF = tackleMergedCells(addDF)

    columns2eng = pd.read_csv('colNames2engNamesAdd.csv')
    columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
    addDF = renameColumns(addDF, columns2eng) # тут заголовок таблицы предварительно чистить не надо

    addDF = addDF[addDF.columns[~addDF.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
    addDF = dropNARows(addDF)
addDF = deleteUselessRows(addDF, year)
clauses2Insert = {'2019': ['263ч.1.2']}
addDF = insertEmptyRows(addDF, year, clauses2Insert)
addDF = cleanClauseCol(addDF)
addDF = clauses2column(addDF)
addDF = nameSeparatedRows(addDF, names)

firstCols = ['name', 'clause', 'part', 'totalConvictedMain']
addDF = rearrangeCols(addDF, firstCols)
    
addDF = keepCombinedRows(addDF, year)
addDF = addMilitaryOfences(addDF)
addDF = addDF[~addDF['name'].str.contains('Составы преступлений, введенные в УК РФ в 2018 году')]

addDF['name'] = addDF['name'].str.replace('^\s{1}', '')
addDF['name'] = addDF['name'].str.replace('\n', '') 
addDF = sortTable(addDF)

Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье


/home/kate/dostoevsky-data/tools.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, val in df.iloc[:,0].str.contains(value2search).iteritems():
/home/kate/dostoevsky-data/tools.py:67: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, col in df.iloc[0:header_rows].iteritems():
/home/kate/dostoevsky-data/tools.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[0:1].fillna(method='ffill', axis='columns', inplace=True)
/home/kate/dostoevsky-data/tools.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье


/home/kate/dostoevsky-data/tools.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, val in df.iloc[:,0].str.contains(value2search).iteritems():
/home/kate/dostoevsky-data/tools.py:67: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, col in df.iloc[0:header_rows].iteritems():
/home/kate/dostoevsky-data/tools.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[0:1].fillna(method='ffill', axis='columns', inplace=True)
/home/kate/dostoevsky-data/tools.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела 

...

/home/kate/dostoevsky-data/tools.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(index=str, columns={col2: col}, inplace=True)
/home/kate/dostoevsky-data/tools.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, val in df.iloc[:,0].str.contains(value2search).iteritems():
/home/kate/dostoevsky-data/tools.py:67: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, col in df.iloc[0:header_rows].iteritems():
/home/kate/dostoevsky-data/tools.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела 

/home/kate/dostoevsky-data/tools.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(index=str, columns={col2: col}, inplace=True)
/home/kate/dostoevsky-data/tools.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(index=str, columns={col2: col}, inplace=True)
/home/kate/dostoevsky-data/tools.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(index=str, columns={col2: col}, inplace=True)
/home/kate/do

Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела 

/home/kate/dostoevsky-data/tools.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(index=str, columns={col2: col}, inplace=True)
/home/kate/dostoevsky-data/tools.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(index=str, columns={col2: col}, inplace=True)
/home/kate/dostoevsky-data/tools.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(index=str, columns={col2: col}, inplace=True)
/home/kate/do

In [26]:
addDF

,name,addAcquittalOffences,addAcquittalPersons,addDismissalOffences,addDismissalOtherOffences,addDismissalOtherPersons,addDismissalPersons,addTotalOffences,addTotalPersons,addUnfitToPleadOffences
0,Убийство,4,4,1,2,2,1,69,67,7
1,Убийство при отягчающих обстоятельствах,58,45,2,7,7,2,136,23,7
2,Убийство матерью новорожденного ребенка,0,0,0,0,0,0,0,0,0
3,"Убийство, совершенное в состоянии аффекта",0,0,0,0,0,0,1,1,0
4,Убийство в состоянии аффекта двух или более лиц,0,0,0,0,0,0,0,0,0
5,Убийство при превышении пределов необходимой о...,0,0,0,0,0,0,7,7,0
6,"Убийство при превышении мер, необходимых для з...",0,0,0,0,0,0,0,0,0
7,Причинение смерти по неосторожности,1,1,0,12,12,0,56,56,1
8,Причинение смерти по неосторожности при отягча...,0,0,0,1,1,0,0,0,0
9,Причинение смерти по неосторожности двум или б...,0,0,0,0,0,0,1,1,0


In [27]:
addDF.columns

Index(['name', 'addAcquittalOffences', 'addAcquittalPersons',
       'addDismissalOffences', 'addDismissalOtherOffences',
       'addDismissalOtherPersons', 'addDismissalPersons', 'addTotalOffences',
       'addTotalPersons', 'addUnfitToPleadOffences'],
      dtype='object')

<a id='data_check'></a>

### Проверка корректности обработки данных

Теперь нужно соединить все три таблицы за год в одну, но сначала проведем несколько проверок.

1) все три таблицы должны быть одной длины.

In [17]:
checkTablesLen(mainDF, addDF, parametersDF)
checkTablesLen(mainDF, addDF)

ОК: длины таблиц № 10.3 и № 10.3.1 совпадают
ОК: длины таблиц № 10.3 и № 10-а совпадают
ОК: длины таблиц № 10.3 и № 10-а совпадают


2) номера статей при построчном сравнении должны совпадать. Однако простое построчное сравнение может нас обмануть.В отчетных формах Судебного департамента могут соседствовать актуальная и утратившая силу части одной и той же статьи с одинаковыми номерами, при этом все примечания к номерам статей мы убрали, оставив их у названий статей/частей. Поэтому построчно имеет смысл сравнивать не только колонку с номерами статей, но и числа в тех столбцах, которые в разных формах должны содержать одинаковую информацию.


Так в форме № 10.3 и № 10.3.1 должны совпадать соответственно колонки "Лишение свободы" и сумма чисел из колонок "До 1 года вкл.", "Свыше 1 до 2 лет вкл.",	"Свыше 2 до 3 лет вкл.", "Свыше 3 до 5 лет вкл.", "Свыше 5 до 8 лет вкл.", "Свыше 8 до 10 лет вкл.", "Свыше 10 до 15 лет вкл.", "Свыше 15 до 20 лет вкл.".

А в формах № 10.3 и № 10-а - колонки "ВСЕГО ОСУЖДЕНО ЛИЦ" и "Число осужденных лиц по основной статье".

На практике так происходит не всегда и числа регулярно расходятся, поэтому приходится ориентироваться не на их полное совпадение, а на величину разницы между ними.

In [18]:
checkNumbersBetweenForms(year, mainDF, addDF, parametersDF)

Не совпадает число осужденных по основной статье. 
Статья:  Воинские преступления 
10.3: 1449 
10-a: nan 
Год: 2019


Теперь можно свести все таблицы за один год в одну, удалив дублирующиеся столбцы, а именно столбцы с номером и частью статьи в формах № 10.3.1 и № 10-а и столбцы с числом осужденных по основному составу преступления в форме № 10-а.

In [19]:
if year > 2010:
    addDF = addDF.drop(['clause', 'part', 'totalConvictedMain'], axis=1)
    parametersDF = parametersDF.drop(['clause', 'part'], axis=1)
    df = pd.concat([mainDF, parametersDF, addDF], axis=1)
else:
    addDF = addDF.drop(['clause', 'part', 'totalConvictedMain'], axis=1)
    df = pd.concat([mainDF, addDF], axis=1)

firstCols = ['name', 'clause', 'part', 'totalConvicted']
df = rearrangeCols(df, firstCols)

In [20]:
df.head()

,name,clause,part,totalConvicted,primaryLifeSentence,primaryImprisonment,primarySuspended,primaryMilitaryDisciplinaryUnit,primaryArrest,primaryRestrain,primaryRestrictionsInMilitaryService,primaryCorrectionalLabour,primaryCommunityService,primaryForcedLabour,primaryDisqualification,primaryFine,primaryOther,exemptionTimeServedFromImprisonment,exemptionTimeServedOther,exemptionAmnestyFromImprisonment,exemptionAmnestyOther,exemptionOtherGroundsFromImprisonment,exemptionOtherGroundsOther,acquittal,dismissalAbsenceOfEvent,...,primaryFine500_1M,primaryFine1M,addFine5,addFine5_25,addFine25_100,addFine100_300,addFine300_500,addFine500_1M,addFine1M,primaryFineSum,addFineSum,dismissalCourtFine5,dismissalCourtFine5_25,dismissalCourtFine25_100,dismissalCourtFine100,dismissalCourtFineSum,addAcquittalOffences,addAcquittalPersons,addDismissalOffences,addDismissalOtherOffences,addDismissalOtherPersons,addDismissalPersons,addTotalOffences,addTotalPersons,addUnfitToPleadOffences
0,Убийство,105,105ч.1,4891,0,4866,22,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,26,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,1,2,2,1,69,67,7
1,Убийство при отягчающих обстоятельствах,105,105ч.2,1306,35,1260,7,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3,0,36,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,58,45,2,7,7,2,136,23,7
2,Убийство матерью новорожденного ребенка,106,106,26,0,14,5,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,20000,0,0,0,0,0,0,0,0,0
3,"Убийство, совершенное в состоянии аффекта",107,107ч.1,38,0,3,0,0,0,24,0,8,0,0,0,0,0,0,2,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,30000,0,0,0,0,0,0,1,1,0
4,Убийство в состоянии аффекта двух или более лиц,107,107ч.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


И привести таблицу в длинную форму.

In [21]:
dfMelted = meltTable(df, year)

In [22]:
dfMelted.head(20)

,year,clause,part,name,parameter,value
0,2019,105,105ч.1,Убийство,totalConvicted,4891
1,2019,105,105ч.2,Убийство при отягчающих обстоятельствах,totalConvicted,1306
2,2019,106,106,Убийство матерью новорожденного ребенка,totalConvicted,26
3,2019,107,107ч.1,"Убийство, совершенное в состоянии аффекта",totalConvicted,38
4,2019,107,107ч.2,Убийство в состоянии аффекта двух или более лиц,totalConvicted,0
5,2019,108,108ч.1,Убийство при превышении пределов необходимой о...,totalConvicted,232
6,2019,108,108ч.2,"Убийство при превышении мер, необходимых для з...",totalConvicted,0
7,2019,109,109ч.1,Причинение смерти по неосторожности,totalConvicted,828
8,2019,109,109ч.2,Причинение смерти по неосторожности при отягча...,totalConvicted,143
9,2019,109,109ч.3,Причинение смерти по неосторожности двум или б...,totalConvicted,37
